<a href="https://colab.research.google.com/github/tkimondox/colab/blob/main/RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP ROTG** : Remote Desktop ROTG to Colab Instance


> **Warning : Not for Cryptocurrency Mining<br></br>** 
>**Why are hardware resources such as T4 GPUs not available to me?** The best available hardware is prioritized for users who use Colaboratory interactively rather than for long-running computations. Users who use Colaboratory for long-running computations may be temporarily restricted in the type of hardware made available to them, and/or the duration that the hardware can be used for. We encourage users with high computational needs to use Colaboratory’s UI with a local runtime. Please note that using Colaboratory for cryptocurrency mining is disallowed entirely, and may result in being banned from using Colab altogether.

Google Colab can give you Instance with 12GB of RAM and GPU for 12 hours (Max.) for Free users. Anyone can use it to perform Heavy Tasks.



In [ ]:
#@title **Create User**
#@markdown Enter Username and Password

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print(f"User created and configured having username `{username}` and password `{password}`")

In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

import os
import subprocess

#@markdown  Visit http://remotedesktop.google.com/headless and copy the command after Authentication

CRP = "" #@param {type:"string"}

#@markdown Enter a Pin (more or equal to 6 digits)
Pin = 123456 #@param {type: "integer"}

#@markdown Autostart Notebook in RDP
Autostart = False #@param {type: "boolean"}


class CRD:
    def __init__(self, user):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish(user)
        print("\nRDP created succesfully move to https://remotedesktop.google.com/access")

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish(user):
        print("Finalizing")
        if Autostart:
            os.makedirs(f"/home/{user}/.config/autostart", exist_ok=True)
            link = "https://colab.research.google.com/github/tkimondox/colab/blob/main/RDP.ipynb"
            colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)
            with open(f"/home/{user}/.config/autostart/colab.desktop", "w") as f:
                f.write(colab_autostart)
            os.system(f"chmod +x /home/{user}/.config/autostart/colab.desktop")
            os.system(f"chown {user}:{user} /home/{user}/.config")

        os.system(f"adduser {user} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {user} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        

        print("Finished Succesfully")


try:
    if CRP == "":
        print("Please enter authcode from the given link")
    elif len(str(Pin)) < 6:
        print("Enter a pin more or equal to 6 digits")
    else:
        CRD(username)
except NameError as e:
    print("'username' variable not found, Create a user first")

In [ ]:
#@title **SSH**

! pip install colab_ssh --upgrade &> /dev/null

#@markdown Choose a method (Agro Recommended)
ssh_method = "Agro" #@param ["Agro", "Ngrok"]


#@markdown Copy authtoken from https://dashboard.ngrok.com/auth (only for ngrok)
ngrokRegion = "us" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"]

def runAgro():
    from colab_ssh import launch_ssh_cloudflared
    launch_ssh_cloudflared(password=password)

def runNgrok():
    from colab_ssh import launch_ssh
    from IPython.display import clear_output

    import getpass
    ngrokToken = getpass.getpass("Enter the ngrokToken: ")

    launch_ssh(ngrokToken, password, region=ngrokRegion)
    clear_output()

    print("ssh", user, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

try:
    user = username
    password = password
except NameError:
    print("No user found, using username and password as 'root'")
    user='root'
    password='root'


if ssh_method == "Agro":
    runAgro()
if ssh_method == "Ngrok":
    runNgrok()

In [ ]:
#@title **Colab Shutdown**

#@markdown To Kill NGROK Tunnel
NGROK = False #@param {type:'boolean'}

#@markdown To Sleep Colab
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if Sleep:
    from time import sleep
    sleep(43200)

# **<font color='red'> ROTG - This version of ROTG is for personal use only </font>**


## **<font color="red">Important note</font> - After doing all the steps, please note that depending on your file size of the mp4 @ mkv files, it may take a while to show up in your Google Drive. Please be patient. Eventually, it will show up in your drive.**
###**<font color="red">For larger files</font> - it may take some time to show up in your Google Drive. Do not rerun the code if it doesn't show up fast. Just keep refreshing your Google Drive page to see if it shows up**
###**<font color="red">For small files</font> - it will show up quicker in your Google Drive. Just wait until it shows up. Just keep refreshing your page to see if it shows up**
###**<font color="red">Update</font> - added some features**
* Multi fragments download (50 frags)
* Add support extract and download subtitle via mpd.
* Add support upload subtitle.
* Add support convert vtt to srt.


#__Mount Google Drive__

In [ ]:
#@title Klik "Play" untuk mount
from google.colab import drive
drive.mount('/content/drive')
print("Mounted!")

#__Install ROTG tools__

In [ ]:
#@title Klik "Play" button untuk install
from IPython.display import clear_output 
!sudo apt update
!sudo apt install aria2
!sudo curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
!sudo chmod a+rx /usr/local/bin/yt-dlp
!sudo curl -L https://github.com/tkimondok/ROTG/raw/main/mp4decrypt -o /usr/local/bin/mp4decrypt
!sudo chmod a+rx /usr/local/bin/mp4decrypt
!sudo apt install ffmpeg
!sudo apt -y install aria2
clear_output()
print("Install successful")

#__1-CLICK RIPPING__

In [ ]:
#@markdown <h2>Extract manifest info</h2>
MPD_URL = 'https://hbogoprod-vod.akamaized.net/video/vod/output/HBOAsia/60c51f4616b55f4d934d7cbc_hbovodhd/dash0/MYS_DOLBY_index.mpd' #@param {type:"string"}
!yt-dlp --no-warnings --allow-u -F "{MPD_URL}"
print("[Note:] Sila copy video ID dan audio ID dan masukkan di kotak dibawah:")

In [ ]:
#@markdown <h2>Downloading video and audio</h2>
from IPython.display import clear_output
VIDEO_ID = 'video_avc1_1' #@param {type:"string"}
AUDIO_ID = 'audio_DOLBY_ec-3' #@param {type:"string"}
VIDEO_OUTPUT = 'default.mp4' #@param {type:"string"}
AUDIO_OUTPUT = 'default.m4a' #@param {type:"string"}
VIDEO_DECRYPT_OUTPUT = 'dec.mp4' #@param {type:"string"}
AUDIO_DECRYPT_OUTPUT = 'dec.m4a' #@param {type:"string"}
KID_KEY = 'bb34dd66328a5597b9ba6d7589d89ffe:54f10655fe0b033e9c820bde8d94618a' #@param {type:"string"}
#@markdown <h4>INFO: Boleh terus save ke Drive @ Team Drive. Hanya perlu masukkan path diikuti output</h4>
FINAL_OUTPUT = 'done.mp4' #@param {type:"string"}
print("Downloading video.....")
!yt-dlp --no-warnings --allow-u -f "{VIDEO_ID}" "{MPD_URL}" -o "{VIDEO_OUTPUT}" -N 60
clear_output()
print("Success")
clear_output()
print("Downloading audio.....")
!yt-dlp --no-warnings --allow-u -f "{AUDIO_ID}" "{MPD_URL}" -o "{AUDIO_OUTPUT}" -N 60
clear_output()
print("Success")
clear_output()
print("Decrypting video.....")
!mp4decrypt --show-progress --key "{KID_KEY}" "{VIDEO_OUTPUT}" "{VIDEO_DECRYPT_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Decrypting audio.....")
!mp4decrypt --show-progress --key "{KID_KEY}" "{AUDIO_OUTPUT}" "{AUDIO_DECRYPT_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Merging video and audio into single file.....")
!ffmpeg -loglevel error -i "{VIDEO_DECRYPT_OUTPUT}" -i "{AUDIO_DECRYPT_OUTPUT}" -c copy "{FINAL_OUTPUT}"
clear_output()
print("Success")
clear_output()
print("Video sudah siap dan ready to play @ share.....\nUntuk masukkan subtitle, silahkan scroll kebawah.")

#__GETTING & DOWNLOADING SUBTITLE FROM MPD__

In [ ]:
#@title List all available subtitle:
!yt-dlp --no-warnings --allow-u --list-subs "{MPD_URL}"

In [ ]:
#@title Download all available subtitles:
!yt-dlp --no-warnings --allow-u -N 32 --skip-download --all-subs "{MPD_URL}" -o subtitle

In [ ]:
#@title Upload own subtitle:
#@markdown *Please enable/allow all cookies from your browser to enable upload button.
from google.colab import files
uploaded = files.upload()

#__CONVERT VTT TO SRT__

In [ ]:
SUBTITLE_INPUT = 'subtitle.MA.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT = 'MA.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT}" "{SUBTITLE_OUTPUT}"
clear_output()
SUBTITLE_INPUT = 'subtitle.EN.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT = 'EN.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT}" "{SUBTITLE_OUTPUT}"
clear_output()
SUBTITLE_INPUT = 'subtitle.CN.vtt' #@param {type:"string"}
SUBTITLE_OUTPUT = 'CN.srt' #@param {type:"string"}
!ffmpeg -loglevel error -i "{SUBTITLE_INPUT}" "{SUBTITLE_OUTPUT}"
clear_output()
print("Done!")

#__MUXING SUBTITLE__

In [ ]:
#@title <h3>Mux 1 subtitle</h3>
SUBTITLE = 'MA.srt' #@param {type:"string"}
LANGUAGE = 'malaysia' #@param {type:"string"}
OUTPUT = 'MOVIES.1080p.WEB-DL.AAC.H264-ROTG.mkv' #@param {type:"string"}
!ffmpeg -i "{FINAL_OUTPUT}" -i "{SUBTITLE}" -map 0 -map 1:s -c copy -metadata:s:s:0 language="{LANGUAGE}" "{OUTPUT}"

In [ ]:
#@markdown <h3>Mux 2 subtitles</h3>
SUBTITLE_01 = 'EN.srt' #@param {type:"string"}
SUBTITLE_02 = 'MA.srt' #@param {type:"string"}
LANGUAGE_01 = 'english' #@param {type:"string"}
LANGUAGE_02 = 'malaysia' #@param {type:"string"}
OUTPUT = 'MOVIES.1080p.WEB-DL.AAC.H264-ROTG.mkv' #@param {type:"string"}
!ffmpeg -i "{FINAL_OUTPUT}" -i "{SUBTITLE_01}" -i "{SUBTITLE_02}" -map 0 -map 1:s -map 2:s -c copy -metadata:s:s:0 language="{LANGUAGE_01}" -metadata:s:s:1 language="{LANGUAGE_02}" "{OUT_PUT}"

In [ ]:
#@markdown <h3>Mux 3 subtitles</h3>
SUBTITLE_01 = 'EN.srt' #@param {type:"string"}
SUBTITLE_02 = 'MA.srt' #@param {type:"string"}
SUBTITLE_03 = 'CN.srt' #@param {type:"string"}
LANGUAGE_01 = 'english' #@param {type:"string"}
LANGUAGE_02 = 'malaysia' #@param {type:"string"}
LANGUAGE_03 = 'china' #@param {type:"string"}
OUTPUT = 'MOVIES.1080p.WEB-DL.AAC.H264-ROTG.mkv' #@param {type:"string"}
!ffmpeg -i "{FINAL_OUTPUT}" -i "{SUBTITLE_01}" -i "{SUBTITLE_02}" -i "{SUBTITLE_03}" -map 0 -map 1:s -map 2:s -map 3:s -c copy -metadata:s:s:0 language="{LANGUAGE_01}" -metadata:s:s:1 language="{LANGUAGE_02}" -metadata:s:s:2 language="{LANGUAGE_03}" "{OUTPUT}"

#__COPY @ MOVE OUTPUT TO NEW PATH__

In [ ]:
#@markdown <h3>Copy output (No Subtitle) from root to new path</h3>
NEW_PATH = '/content/drive/MyDrive' #@param {type:"string"}
!cp -r "{FINAL_OUTPUT}" "{NEW_PATH}"

In [ ]:
#@markdown <h3>Copy output (Muxed subtitle) from root to new path</h3>
NEW_PATH = '/content/drive/MyDrive' #@param {type:"string"}
!cp -r "{OUTPUT}" "{NEW_PATH}"

In [ ]:
#@markdown <h3>Move output (No Subtitle) from root to new path</h3>
NEW_PATH = '/content/drive/MyDrive' #@param {type:"string"}
!mv "{FINAL_OUTPUT}" "{NEW_PATH}"

In [ ]:
#@markdown <h3>Move output (Muxed subtitle) from root to new path</h3>
NEW_PATH = '/content/drive/MyDrive' #@param {type:"string"}
!cp -r "{OUTPUT}" "{NEW_PATH}"

#__Upload to Google Drive__

In [ ]:
#@title Download
LINK_FILE = "http://139.59.255.159/7.Hari.Mencintaiku.S03E12.1080p.IQ.WEB-DL.AAC.H.264-NOGRP.mp4" #@param {type:"string"}
#@markdown Path support Google Drive dan SharedDrive @ Team Drive (/content/drive/Shareddrives/FOLDER)
PATH = "/content/drive/Shareddrives/ROTG" #@param {type:"string"}
print("Downloading.....")
!aria2c --file-allocation=none -x16 "{LINK_FILE}" --dir="{PATH}"
print("Done!")